ВЫГРУЗКА СЕРВИСОВ ИЗ ОСМ

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import osmnx as ox

In [ ]:
# Загрузка границ города
city_name = "Санкт-Петербург, Россия"
gdf = ox.geocode_to_gdf(city_name)

output_file = "spb_boundary.geojson"
gdf.to_file(output_file, driver='GeoJSON')

print(f"Границы города СПБ сохранены в {output_file}")

In [ ]:
import os
import geopandas as gpd
import osmnx as ox

# Указываем путь до файла с границами территории (до файла, не до папки!)
input_geojson = r"spb_boundary.geojson"   # поправьте на своё

# Загружаем полигон из файла geojson
polygon = gpd.read_file(input_geojson)
district_name = '_' + os.path.splitext(os.path.basename(input_geojson))[0]

# Преобразуем полигон в формат, используемый osmnx
polygon = polygon.to_crs(epsg=4326)
polygon_geom = polygon['geometry'].unary_union

# Функция для загрузки данных по тегу
def download_osm_data(tag, polygon):
    try:
        data = ox.geometries_from_polygon(polygon, tags=tag)
        if data.empty:
            raise ValueError("No data found for tag: {}".format(tag))
        return data
    except Exception as e:
        print(f"Error downloading data for tag {tag}: {e}")
        return gpd.GeoDataFrame()

# Функция для преобразования списков в строки
def convert_lists_to_strings(gdf):
    for col in gdf.columns:
        if gdf[col].dtype == 'object':
            gdf[col] = gdf[col].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
    return gdf

# Функция для удаления атрибутов, кроме указанных
def remove_unwanted_attributes(gdf, keep_attributes):
    keep_attributes.append('geometry')
    return gdf[[col for col in gdf.columns if col in keep_attributes]]

# Теги для загрузки
tags = {
    'multifunctional_center': {'office': 'goverment'},
    'kindergarten': {'amenity': 'kindergarten'},
    'school': {'amenity': 'school'},
    'sports_centre': {'leisure': 'sports_centre'},
    'summer_camp': {'leisure': 'summer_camp'},
    'college': {'amenity': 'college'},
    'university': {'amenity': 'university'},

    'polyclinic': {'amenity': 'clinic'},
    'dentist': {'amenity': 'dentist'},
    'sanatorium': {'amenity': 'sanatorium'},
    'hospital': {'amenity': 'hospital'},
    'pharmacy': {'amenity': 'pharmacy'},
    'hospice': {'social_facility': 'hospice'},
    'ambulance_station': {'emergency': 'ambulance_station'},
    'mortuary': {'amenity': 'mortuary'},
    'nursing_home': {'social_facility': 'nursing_home'},

    'community_centre': {'amenity': 'community_centre'},
    'recruitment': {'government': 'recruitment'},
    'library': {'amenity': 'library'},
    'museum': {'tourism': 'museum'},
    'theatre': {'amenity': 'theatre'},
    'botanical_garden': {'garden:type': 'botanical'},
    'concert_hall': {'amenity': 'concert_hall'},
    'zoo': {'tourism': 'zoo'},
    'cinema': {'amenity': 'cinema'},
    'mall': {'shop': 'mall'},
    'water_park': {'leisure': 'water_park'},
    'stadium': {'leisure': 'stadium'},
    'ice_rink': {'leisure': 'ice_rink'},
    'cafe': {'amenity': 'cafe'},
    'restaurant': {'amenity': 'restaurant'},
    'bar': {'amenity': 'bar'},
    'cafeteria': {'fast_food': 'cafeteria'},
    'bakery': {'shop': 'bakery'},

    'park': {'leisure': 'park'},
    'forest': {'landuse': 'forest'},
    'protected_area': {'boundary': 'protected_area'},
    'nature_reserve': {'leisure': 'nature_reserve'},
    'grass': {'landuse': 'grass'},
    'meadow': {'landuse': 'meadow'},
    'dog_park': {'leisure': 'dog_park'},

    'river': {'waterway': 'river'},
    'stream': {'waterway': 'stream'},
    'bay': {'natural': 'bay'},
    'water_drain': {'water': 'drain'},
    'water_wastewater': {'water': 'wastewater'},
    'water_reservoir': {'water': 'reservoir'},
    'water_channel': {'water': 'channel'},
    'water_canal': {'water': 'canal'},
    'water_lock': {'water': 'lock'},
    'water_basin': {'water': 'basin'},
    'water_oxbow': {'water': 'oxbow'},
    'water_lake': {'water': 'lake'},
    'water_pond': {'water': 'pond'},
    'strait': {'natural': 'strait'},

    'pitch': {'leisure': 'pitch'},
    'swimming_pool': {'leisure': 'swimming_pool'},
    'sports_hall': {'leisure': 'sports_hall'},
    'climbing': {'sport': 'climbing'},
    'playground': {'leisure': 'playground'},
    'theme_park': {'tourism': 'theme_park'},
    'skateboard': {'sport': 'skateboard'},

    'train_station': {'building': 'train_station'},
    'railway_station': {'railway': 'station'},
    'aerodrome': {'aeroway': 'aerodrome'},
    'fuel': {'amenity': 'fuel'},
    'parking': {'amenity': 'parking'},
    'bus_stop': {'highway': 'bus_stop'},

    'shop_supermarket': {'shop': 'supermarket'},
    'convenience': {'shop': 'convenience'},
    'shop_marketplace': {'amenity': 'marketplace'},
    'shop_houseware': {'shop': 'houseware'},
    'shop_shoes': {'shop': 'shoes'},
    'shop_clothes': {'shop': 'clothes'},
    'shop_electronics': {'shop': 'electronics'},
    'shop_electrical': {'shop': 'electrical'},
    'shop_books': {'shop': 'books'},
    'baby_goods': {'shop': 'baby_goods'},
    'shop_sports': {'shop': 'sports'},

    'police': {'amenity': 'police'},
    'fire_station': {'amenity': 'fire_station'},
    'post_office': {'amenity': 'post_office'},
    'outpost': {'shop': 'outpost'},
    'bank': {'amenity': 'bank'},
    'atm': {'amenity': 'atm'},

    'lawyer': {'office': 'lawyer'},
    'notary': {'office': 'lawyer', 'lawyer': 'notary'},
    'hairdresser': {'shop': 'hairdresser'},
    'shop_beauty': {'shop': 'beauty'},
    'public_bath': {'amenity': 'public_bath'},
    'veterinary': {'amenity': 'veterinary'},
    'shop_pet': {'shop': 'pet'},

    'hotel': {'tourism': 'hotel'},
    'hostel': {'tourism': 'hostel'},
    'resort': {'leisure': 'resort'},
    'beach_resort': {'leisure': 'beach_resort'},
    'memorial': {'historic': 'memorial'},
    'monument': {'historic': 'monument'},
    'church': {'building': 'church'},
    'cemetery': {'landuse': 'cemetery'},

    'substation': {'power': 'substation'},
    'hydro': {'generator:source': 'hydro'},
    'heating_station': {'industrial': 'heating_station'},
    'generator': {'power': 'generator'},
    'water_works': {'man_made': 'water_works'},
    'pumping_station': {'man_made': 'pumping_station'},
    'wastewater_plant': {'man_made': 'wastewater_plant'},
    'water_tower': {'man_made': 'water_tower'},
    'military': {'landuse': 'military'},
    'industrial': {'landuse': 'industrial'}
}

# Определение необходимых типов геометрий для каждого тега
geometry_types = {
    'multifunctional_center': ['Point'],
    'kindergarten': ['Polygon', 'MultiPolygon'],
    'school': ['Polygon', 'MultiPolygon'],
    'community_centre': ['Polygon', 'MultiPolygon'],
    'sports_centre': ['Polygon', 'MultiPolygon'],
    'summer_camp': ['Polygon', 'MultiPolygon'],
    'college': ['Polygon', 'MultiPolygon'],
    'university': ['Polygon', 'MultiPolygon'],
    'polyclinic': ['Polygon', 'MultiPolygon'],
    'dentist': ['Polygon', 'MultiPolygon'],
    'sanatorium': ['Polygon', 'MultiPolygon'],
    'hospital': ['Polygon', 'MultiPolygon'],
    'pharmacy': ['Polygon', 'MultiPolygon'],
    'hospice': ['Polygon', 'MultiPolygon'],
    'ambulance_station': ['Polygon', 'MultiPolygon'],
    'mortuary': ['Polygon', 'MultiPolygon'],
    'nursing_home': ['Polygon', 'MultiPolygon'],
    'recruitment': ['Polygon', 'MultiPolygon'],
    'library': ['Polygon', 'MultiPolygon'],
    'museum': ['Polygon', 'MultiPolygon'],
    'theatre': ['Polygon', 'MultiPolygon'],
    'botanical_garden': ['Polygon', 'MultiPolygon'],
    'concert_hall': ['Polygon', 'MultiPolygon'],
    'zoo': ['Polygon', 'MultiPolygon'],
    'cinema': ['Polygon', 'MultiPolygon'],
    'mall': ['Polygon', 'MultiPolygon'],
    'water_park': ['Polygon', 'MultiPolygon'],
    'stadium': ['Polygon', 'MultiPolygon'],
    'ice_rink': ['Polygon', 'MultiPolygon'],
    'cafe': ['Polygon', 'MultiPolygon'],
    'restaurant': ['Polygon', 'MultiPolygon'],
    'bar': ['Polygon', 'MultiPolygon'],
    'cafeteria': ['Polygon', 'MultiPolygon'],
    'bakery': ['Polygon', 'MultiPolygon'],
    'park': ['Polygon', 'MultiPolygon'],
    'forest': ['Polygon', 'MultiPolygon'],
    'protected_area': ['Polygon', 'MultiPolygon'],
    'nature_reserve': ['Polygon', 'MultiPolygon'],
    'grass': ['Polygon', 'MultiPolygon'],
    'meadow': ['Polygon', 'MultiPolygon'],
    'beach_resort': ['Polygon', 'MultiPolygon'],
    'river': ['LineString', 'MultiLineString'],
    'stream': ['Polygon', 'MultiPolygon'],
    'bay': ['Polygon', 'MultiPolygon'],
    'water_drain': ['Polygon', 'MultiPolygon'],
    'water_wastewater': ['Polygon', 'MultiPolygon'],
    'water_reservoir': ['Polygon', 'MultiPolygon'],
    'water_channel': ['Polygon', 'MultiPolygon'],
    'water_canal': ['Polygon', 'MultiPolygon'],
    'water_lock': ['Polygon', 'MultiPolygon'],
    'water_basin': ['Polygon', 'MultiPolygon'],
    'water_oxbow': ['Polygon', 'MultiPolygon'],
    'water_lake': ['Polygon', 'MultiPolygon'],
    'water_pond': ['Polygon', 'MultiPolygon'],
    'strait': ['Polygon', 'MultiPolygon'],
    'pitch': ['Polygon', 'MultiPolygon'],
    'swimming_pool': ['Polygon', 'MultiPolygon'],
    'sports_hall': ['Polygon', 'MultiPolygon'],
    'climbing': ['Polygon', 'MultiPolygon'],
    'playground': ['Polygon', 'MultiPolygon'],
    'theme_park': ['Polygon', 'MultiPolygon'],
    'skateboard': ['Polygon', 'MultiPolygon'],
    'police': ['Polygon', 'MultiPolygon'],
    'fire_station': ['Polygon', 'MultiPolygon'],
    'train_station': ['Polygon', 'MultiPolygon'],
    'railway_station': ['Polygon', 'MultiPolygon'],
    'aerodrome': ['Polygon', 'MultiPolygon'],
    'fuel': ['Polygon', 'MultiPolygon'],
    'parking': ['Polygon', 'MultiPolygon'],
    'bus_stop': ['Polygon', 'MultiPolygon'],
    'shop_supermarket': ['Polygon', 'MultiPolygon'],
    'convenience': ['Polygon', 'MultiPolygon'],
    'shop_marketplace': ['Polygon', 'MultiPolygon'],
    'shop_houseware': ['Polygon', 'MultiPolygon'],
    'shop_shoes': ['Polygon', 'MultiPolygon'],
    'shop_clothes': ['Polygon', 'MultiPolygon'],
    'shop_electronics': ['Polygon', 'MultiPolygon'],
    'shop_electrical': ['Polygon', 'MultiPolygon'],
    'shop_books': ['Polygon', 'MultiPolygon'],
    'baby_goods': ['Polygon', 'MultiPolygon'],
    'shop_sports': ['Polygon', 'MultiPolygon'],
    'post_office': ['Polygon', 'MultiPolygon'],
    'outpost': ['Polygon', 'MultiPolygon'],
    'bank': ['Polygon', 'MultiPolygon'],
    'atm': ['Polygon', 'MultiPolygon'],
    'lawyer': ['Polygon', 'MultiPolygon'],
    'notary': ['Polygon', 'MultiPolygon'],
    'hairdresser': ['Polygon', 'MultiPolygon'],
    'shop_beauty': ['Polygon', 'MultiPolygon'],
    'public_bath': ['Polygon', 'MultiPolygon'],
    'veterinary': ['Polygon', 'MultiPolygon'],
    'shop_pet': ['Polygon', 'MultiPolygon'],
    'dog_park': ['Polygon', 'MultiPolygon'],
    'hotel': ['Polygon', 'MultiPolygon'],
    'hostel': ['Polygon', 'MultiPolygon'],
    'resort': ['Polygon', 'MultiPolygon'],
    'memorial': ['Polygon', 'MultiPolygon'],
    'monument': ['Polygon', 'MultiPolygon'],
    'church': ['Polygon', 'MultiPolygon'],
    'cemetery': ['Polygon', 'MultiPolygon'],
    'substation': ['Point'],
    'hydro': ['Point'],
    'heating_station': ['Point'],
    'generator': ['Point'],
    'water_works': ['Point'],
    'pumping_station': ['Point'],
    'wastewater_plant': ['Point'],
    'water_tower': ['Point'],
    'military': ['Polygon', 'MultiPolygon'],
    'industrial': ['Polygon', 'MultiPolygon']
}

# Функция для обрезки слоя по границам полигона
def clip_layer_to_polygon(layer_data, polygon_geom):
    try:
        return gpd.clip(layer_data, polygon_geom)
    except Exception as e:
        print(f"Error clipping layer: {e}")
        return gpd.GeoDataFrame()

# Загрузка данных OSM и сохранение слоев
layers = {name: download_osm_data(tag, polygon_geom) for name, tag in tags.items()}

# List of layers that should NOT be converted to centroids
no_centroid_conversion = [
    'forest', 'protected_area', 'nature_reserve', 'grass', 'meadow',
    'beach_resort', 'stream', 'bay', 'water_drain', 'water_wastewater',
    'water_reservoir', 'water_channel', 'water_canal', 'water_lock',
    'water_basin', 'water_oxbow', 'water_lake', 'water_pond', 'strait',
    'aerodrome', 'parking', 'military', 'industrial', 'cemetery'
]

# Проставление значения type для каждого слоя
for layer_name, layer_data in layers.items():
    if not layer_data.empty:
        # Фильтрация по типу геометрии
        geom_types = geometry_types.get(layer_name, [])
        layer_data = layer_data[layer_data.geometry.type.isin(geom_types)]

        if not layer_data.empty:
            # Обрезка слоя по границам полигона
            layer_data = clip_layer_to_polygon(layer_data, polygon_geom)

            if not layer_data.empty:
                # Удаление всех атрибутов, кроме указанных
                keep_attributes = ['name', 'name:ru', 'osm_id', 'addr:street', 'addr:housenumber']
                layer_data = remove_unwanted_attributes(layer_data, keep_attributes)

                # Преобразование списков в строки
                layer_data = convert_lists_to_strings(layer_data)

                # Преобразование в GeoDataFrame, если это не так
                if not isinstance(layer_data, gpd.GeoDataFrame):
                    layer_data = gpd.GeoDataFrame(layer_data, geometry='geometry')

                # Перевод объектов 'Polygon' и 'MultiPolygon' в центроиды, если не в исключении
                if layer_data.geometry.type.isin(['Polygon', 'MultiPolygon']).any() and layer_name not in no_centroid_conversion:
                    layer_data['geometry'] = layer_data.geometry.centroid
                    print(f"Converted geometries to centroids for layer '{layer_name}'")

                layers[layer_name] = layer_data
            else:
                print(f"Слой '{layer_name}' пуст после обрезки и не будет сохранен")
                layers[layer_name] = gpd.GeoDataFrame()
        else:
            print(f"Слой '{layer_name}' пуст после фильтрации по геометрии")
            layers[layer_name] = gpd.GeoDataFrame()
    else:
        print(f"Слой '{layer_name}' пуст и не будет сохранен")
        layers[layer_name] = gpd.GeoDataFrame()

# Объединение магазинов обуви и одежды
if 'shop_shoes' in layers and 'shop_clothes' in layers:
    if not layers['shop_shoes'].empty and not layers['shop_clothes'].empty:
        layers['shop_shoes_clothes'] = pd.concat([layers['shop_shoes'], layers['shop_clothes']], ignore_index=True)
        print("Слои 'shop_shoes' и 'shop_clothes' успешно объединены в 'shop_shoes_clothes'")
    else:
        print("Один из слоев 'shop_shoes' или 'shop_clothes' пуст и не может быть объединен")
else:
    print("Слои 'shop_shoes' или 'shop_clothes' отсутствуют")

# Объединение электроники
if 'shop_electronics' in layers and 'shop_electrical' in layers:
    if not layers['shop_electronics'].empty and not layers['shop_electrical'].empty:
        layers['shop_shoes_clothes'] = pd.concat([layers['shop_electronics'], layers['shop_electrical']], ignore_index=True)
        print("Слои 'shop_electronics' и 'shop_electrical' успешно объединены в 'shop_electronic'")
    else:
        print("Один из слоев 'shop_electronics' или 'shop_electrical' пуст и не может быть объединен")
else:
    print("Слои 'shop_electronics' или 'shop_electrical' отсутствуют")

# Папка для сохранения результатов
output_dir = r"сервисы СПБ" # путь до папки, куда всё сохранится
os.makedirs(output_dir, exist_ok=True)

# Сохранение каждого слоя в отдельный файл
failed_layers = []
for layer_name, layer_data in layers.items():
    if not layer_data.empty:
        # Формирование имени файла без использования имени исходного файла
        file_path = os.path.join(output_dir, f"{layer_name}.geojson")
        try:
            layer_data.to_file(file_path, driver='GeoJSON')
            print(f"Слой '{layer_name}' сохранен в файл {file_path}")
        except ValueError as e:
            print(f"Ошибка при сохранении слоя '{layer_name}': {e}")
            failed_layers.append(layer_name)
    else:
        failed_layers.append(layer_name)

# Вывод списка слоев, которые не удалось сохранить
if failed_layers:
    print("Следующие слои не удалось выгрузить или сохранить:")
    for failed_layer in failed_layers:
        print(f"- {failed_layer}")
else:
    print("Все слои успешно выгружены и сохранены.")
